In [1]:
%%capture
%pip install torchvision
%pip install torchsummary
%pip install torchviz
%pip install hiddenlayer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/Othercomputers/My Computer/Masters_Staff/trimester_3/Deep_Learning

Mounted at /content/drive
/content/drive/Othercomputers/My Computer/Masters_Staff/trimester_3/Deep_Learning


In [3]:
import torch
import torchvision
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import torchvision.transforms as transforms
import torch.nn.functional as F

from torch.utils.data import DataLoader, Subset
from torchsummary import summary
from torch.autograd import Variable
from torchvision import datasets, transforms

In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Jun 28 16:36:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Dataset Loading

In [5]:
#model params
params = {
    'batch_size':32,
    'mul': 30,
    'num_classes':10,
    'channels':3,
    'img_dim':32,
    'device':torch.device('cuda') if torch.cuda.is_available() else 'cpu',
    'fliplr':True,
    'num_epochs':50,
    'decay_epoch':50,
    'lr':0.00002,    #learning rate for generator
    'beta1':0.5 ,    #beta1 for Adam optimizer
    'beta2':0.999 ,  #beta2 for Adam optimizer
    'lambdaA':10 ,   #lambdaA for cycle loss
    'lambdaB':10  ,  #lambdaB for cycle loss
}


In [6]:
# Function to load and preprocess datasets
def load_dataset(dataset_cls, split, root, download, normalize=False, subset_size=None, batch_size=params['batch_size']):
    """
      Load and preprocess a dataset.
      :param dataset_cls: Dataset class (e.g., datasets.MNIST, datasets.SVHN)
      :param split: Data split ('train' or 'test')
      :param root: Root directory for dataset storage/download
      :param download: Flag to download the dataset if not present
      :param normalize: Flag to normalize the dataset
      :param subset_size: Size of the subset to use (if specified)
      :param batch_size: Batch size for DataLoader
      :return: DataLoader object
    """
    if dataset_cls == datasets.MNIST:
        dataset = dataset_cls(root=root, train=(split=='train'), download=download, transform=transforms.ToTensor())
    else:
        dataset = dataset_cls(root=root, split=split, download=download, transform=transforms.ToTensor())

    if normalize:
        # mean, std = compute_mean_and_std(dataset)

        if dataset_cls == datasets.MNIST:

            print(f"Dataset is MNIST: {dataset_cls}")
            transform = transforms.Compose([
                transforms.Resize(( params['img_dim'],params['img_dim']) ),
                transforms.Grayscale(params['channels']),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5]*params['channels'], std=[0.5]*params['channels'])
            ])\

        else:
            print(f"Dataset is SVHN: {dataset_cls}")
            transform = transforms.Compose([
                transforms.Resize(( params['img_dim'],params['img_dim'] )),
                transforms.Grayscale(params['channels']),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5]*params['channels'], std=[0.5]*params['channels'])
            ])

        # Apply the computed normalization
        dataset.transform = transform

    if subset_size:
        indices = np.random.choice(len(dataset), subset_size, replace=False)
        print(dataset.transform)
        dataset = Subset(dataset, indices)

    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True if split == 'train' else False, num_workers=1)
    return loader

# Load MNIST datasets
mnist_trainloader = load_dataset(datasets.MNIST, 'train', './data/', True, normalize=True,subset_size=params['batch_size']*params['mul'] )
mnist_testloader = load_dataset(datasets.MNIST, 'test', './data/', True, normalize=True)

# Load SVHN datasets
svhn_trainloader = load_dataset(datasets.SVHN, 'train', './data/', True, normalize=True,subset_size=params['batch_size']*params['mul'])
svhn_testloader = load_dataset(datasets.SVHN, 'test', './data/', True, normalize=True)

# Optionally, load the extra SVHN dataset if needed
# svhn_extraloader = load_dataset(datasets.SVHN, 'extra', './data/', True, normalize=True, subset_size=20000)

# Verify the sizes of the datasets
print(f'MNIST train subset size: {len(mnist_trainloader.dataset)}')
# print(f'SVHN train subset size: {len(svhn_trainloader.dataset)}')
# print(f'SVHN extra size: {len(svhn_extraloader.dataset)}')



Dataset is MNIST: <class 'torchvision.datasets.mnist.MNIST'>
Compose(
    Resize(size=(32, 32), interpolation=bilinear, max_size=None, antialias=True)
    Grayscale(num_output_channels=3)
    ToTensor()
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
)
Dataset is MNIST: <class 'torchvision.datasets.mnist.MNIST'>
Using downloaded and verified file: ./data/train_32x32.mat
Dataset is SVHN: <class 'torchvision.datasets.svhn.SVHN'>
Compose(
    Resize(size=(32, 32), interpolation=bilinear, max_size=None, antialias=True)
    Grayscale(num_output_channels=3)
    ToTensor()
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
)
Using downloaded and verified file: ./data/test_32x32.mat
Dataset is SVHN: <class 'torchvision.datasets.svhn.SVHN'>
MNIST train subset size: 960


In [7]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define the transformations (same as during training)
transform = transforms.Compose([
    transforms.Grayscale(3),  # Ensure images are single-channel grayscale
    transforms.ToTensor(),
    transforms.Normalize([0.5]*params['channels'], [0.5]*params['channels'])  # Assuming normalization parameters used during training
])

# Define the path to the transformed images
transformed_images_path = 'transformed_mnist_train'

# Create the dataset
transformed_dataset = datasets.ImageFolder(root=transformed_images_path, transform=transform)

# Create the DataLoader
transformed_dataloader = DataLoader(transformed_dataset, batch_size=params['batch_size'], shuffle=True, num_workers=8)

### Baseline Model

Our baseline model here is a simple CNN.

In [8]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Use a dummy input to determine the size of the flattened features
        dummy_input = torch.randn(1, 3, 32, 32)  # Change the input size if necessary
        dummy_output = self.feature_extractor(dummy_input)
        flattened_size = dummy_output.view(-1).size(0)

        # Now use the calculated flattened size for the classifier
        self.classifier = nn.Sequential(
            nn.Linear(flattened_size, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        features = self.feature_extractor(x)
        features = features.view(features.size(0), -1)
        output = self.classifier(features)
        return output

class DomainDiscriminator(nn.Module):
    def __init__(self, feature_size):
        super(DomainDiscriminator, self).__init__()
        self.domain_classifier = nn.Sequential(
            nn.Linear(feature_size, 512),
            nn.ReLU(),
            nn.Linear(512, 2)  # Two classes: source and target
        )

    def forward(self, x):
        output = self.domain_classifier(x)
        return output

# Initialize the model and print the flattened feature size
model = SimpleCNN()
print(model)


SimpleCNN(
  (feature_extractor): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=2048, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=10

In [9]:
feature_extractor = SimpleCNN().to(params['device'])
flattened_size = feature_extractor.classifier[0].in_features  # Get the input feature size of the first Linear layer
domain_discriminator = DomainDiscriminator(flattened_size).to(params['device'])

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(list(feature_extractor.parameters()) + list(domain_discriminator.parameters()), lr=0.00001)


#### Baseline model Training

In [10]:
def train(model, domain_model, source_loader, target_loader, criterion, optimizer, num_epochs=20):
    """
      Train the models.
      :param model: Main model
      :param domain_model: Domain discriminator model
      :param source_loader: DataLoader for source data
      :param target_loader: DataLoader for target data
      :param criterion: Loss function
      :param optimizer: Optimizer for model parameters
      :param num_epochs: Number of training epochs
      :return: None
    """
    model.train()
    domain_model.train()

    for epoch in range(num_epochs):
        for (source_data, source_labels), (target_data, _) in zip(source_loader, target_loader):
            source_data, source_labels = source_data.to(params['device']), source_labels.to(params['device'])
            target_data = target_data.to(params['device'])

            # Train feature extractor and classifier on source data
            optimizer.zero_grad()
            source_features = model.feature_extractor(source_data)
            source_output = model.classifier(source_features.view(source_features.size(0), -1))
            source_loss = criterion(source_output, source_labels)
            source_loss.backward()
            optimizer.step()

            # Train domain discriminator on source and target data
            optimizer.zero_grad()
            domain_labels = torch.cat((torch.zeros(source_data.size(0)), torch.ones(target_data.size(0)))).long().to(params['device'])
            combined_data = torch.cat((source_data, target_data))
            combined_features = model.feature_extractor(combined_data)
            domain_output = domain_model(combined_features.view(combined_features.size(0), -1))
            domain_loss = criterion(domain_output, domain_labels)
            domain_loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Source Loss: {source_loss.item():.4f}, Domain Loss: {domain_loss.item():.4f}')

# Train the model
train(feature_extractor, domain_discriminator, mnist_trainloader, svhn_trainloader, criterion, optimizer)


Epoch [1/20], Source Loss: 2.0953, Domain Loss: 0.1693
Epoch [2/20], Source Loss: 1.9892, Domain Loss: 0.0528
Epoch [3/20], Source Loss: 1.6882, Domain Loss: 0.0260
Epoch [4/20], Source Loss: 1.6248, Domain Loss: 0.0136
Epoch [5/20], Source Loss: 1.3863, Domain Loss: 0.0111
Epoch [6/20], Source Loss: 1.3057, Domain Loss: 0.0093
Epoch [7/20], Source Loss: 1.0706, Domain Loss: 0.0064
Epoch [8/20], Source Loss: 1.1067, Domain Loss: 0.0043
Epoch [9/20], Source Loss: 1.0098, Domain Loss: 0.0047
Epoch [10/20], Source Loss: 0.8393, Domain Loss: 0.0033
Epoch [11/20], Source Loss: 0.7522, Domain Loss: 0.0035
Epoch [12/20], Source Loss: 0.6173, Domain Loss: 0.0025
Epoch [13/20], Source Loss: 0.6056, Domain Loss: 0.0026
Epoch [14/20], Source Loss: 0.5125, Domain Loss: 0.0022
Epoch [15/20], Source Loss: 0.5702, Domain Loss: 0.0030
Epoch [16/20], Source Loss: 0.5472, Domain Loss: 0.0013
Epoch [17/20], Source Loss: 0.4248, Domain Loss: 0.0022
Epoch [18/20], Source Loss: 0.2203, Domain Loss: 0.0019
E

In [11]:
def evaluate(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, labels in loader:
            data, labels = data.to(params['device']), labels.to(params['device'])
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

source_accuracy = evaluate(feature_extractor, mnist_testloader, params['device'])
target_accuracy = evaluate(feature_extractor, svhn_testloader, params['device'])
print(f'Source (MNIST) Accuracy: {source_accuracy:.2f}%')
print(f'Target (SVHN) Accuracy: {target_accuracy:.2f}%')


Source (MNIST) Accuracy: 93.00%
Target (SVHN) Accuracy: 20.31%


### VGG16 architexture

In [12]:
# Define the VGG-16 architecture without pretrained weights
class VGG16Custom(nn.Module):
    def __init__(self, num_classes=params['num_classes']):
        super(VGG16Custom, self).__init__()
        # Load the VGG-16 model
        self.features = models.vgg16(weights=None).features
        # Modify the first convolutional layer to accept 1-channel input
        self.features[0] = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        # Add a custom classifier (fully connected layers)
        self.classifier = nn.Sequential(
            nn.Linear(512 * 1 * 1, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 512 * 1 * 1)
        x = self.classifier(x)
        return x

In [13]:
def CORAL(source, target):
    d = source.data.shape[1]

    # source covariance
    xm = torch.mean(source, 0, keepdim=True) - source
    xc = xm.t() @ xm

    # target covariance
    xmt = torch.mean(target, 0, keepdim=True) - target
    xct = xmt.t() @ xmt

    # frobenius norm between source and target
    loss = torch.mean(torch.mul((xc - xct), (xc - xct)))
    loss = loss/(4*d*d)

    return loss


In [21]:
class DeepCORAL(nn.Module):
    def __init__(self, num_classes=params['num_classes']):
        super(DeepCORAL, self).__init__()
        self.sharedNet = VGG16Custom()
        self.fc = nn.Linear(512, num_classes)

        # initialize according to CORAL paper experiment
        self.fc.weight.data.normal_(0, 0.005)

    def forward(self, source, target):
        source = self.sharedNet(source)
        source = self.fc(source)

        target = self.sharedNet(target)
        target = self.fc(target)
        return source, target

# Function to load pretrained weights into the DeepCORAL model
def load_pretrained(model, pretrained_path):
    pretrained_vgg = torch.load(pretrained_path)
    model_dict = model.state_dict()
    pretrained_dict = {k: v for k, v in pretrained_vgg.items() if k in model_dict and not k.startswith('classifier.6')}
    model_dict.update(pretrained_dict)
    model.load_state_dict(model_dict)
    return model

# Example of loading pretrained weights into DeepCORAL
pretrained_path = 'model_checkpoints/3channeled_vgg16_mnist.pth'  # Specify your pre-trained VGG-16 model path
deepcoral = DeepCORAL(num_classes=10)
deepcoral = load_pretrained(deepcoral.sharedNet, pretrained_path)
print(deepcoral)


VGG16Custom(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, di

In [22]:
# Move model to device
deepcoral.to(params['device'])

optimizer = optim.Adam(deepcoral.parameters(), lr=0.00002)

In [30]:
source_trainloader = mnist_trainloader
target_trainloader = svhn_trainloader

# Training function
def train(model, optimizer, epoch, _lambda, params):
    model.train()
    result = []
    source, target = list(iter(source_trainloader)), list(iter(target_trainloader))
    train_steps = min(len(source), len(target))

    for batch_idx in range(train_steps):
        source_data, source_label = source[batch_idx]
        target_data, _ = target[batch_idx]

        source_data, source_label = Variable(source_data.to(params['device'])), Variable(source_label.to(params['device']))
        target_data = Variable(target_data.to(params['device']))

        optimizer.zero_grad()
        out1, out2 = model(source_data, target_data)
        classification_loss = F.cross_entropy(out1, source_label)
        coral_loss = CORAL(out1, out2)  # Assuming CORAL is defined elsewhere
        total_loss = _lambda * coral_loss + classification_loss
        total_loss.backward()
        optimizer.step()

        result.append({
            'epoch': epoch,
            'step': batch_idx + 1,
            'total_steps': train_steps,
            'lambda': _lambda,
            'coral_loss': coral_loss.item(),
            'classification_loss': classification_loss.item(),
            'total_loss': total_loss.item()
        })

        print('Train Epoch: {:2d} [{:2d}/{:2d}]\t'
              'Lambda: {:.4f}, Class: {:.6f}, CORAL: {:.6f}, Total_Loss: {:.6f}'.format(
            epoch, batch_idx + 1, train_steps, _lambda,
            classification_loss.item(), coral_loss.item(), total_loss.item()))

    return result

# Testing function
def test(model, dataset_loader, epoch, params):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in dataset_loader:
            data, target = data.to(params['device']), target.to(params['device'])
            out = model(data)
            test_loss += F.cross_entropy(out, target, reduction='sum').item()
            pred = out.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(dataset_loader.dataset)
    accuracy = 100. * correct / len(dataset_loader.dataset)

    print('\nTest Epoch: {} Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        epoch, test_loss, correct, len(dataset_loader.dataset), accuracy))

    return {
        'epoch': epoch,
        'average_loss': test_loss,
        'correct': correct,
        'total': len(dataset_loader.dataset),
        'accuracy': accuracy
    }

# Main training and testing loop
args = {
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    # 'transformed_dataloader': transformed_dataloader,
    'svhn_trainloader': svhn_trainloader,
    'mnist_trainloader': mnist_testloader,
}

best_accuracy = 0.0

for epoch in range(1, 5 + 1):
    train_results = train(deepcoral, optimizer, epoch, _lambda=0.5, params=args)
    test_results = test(deepcoral, svhn_testloader, epoch, params=args)
    print(f"Test Epoch: {epoch}, Accuracy: {test_results['accuracy']:.2f}%")

    # Save the model if the accuracy improves
    if test_results['accuracy'] > best_accuracy:
        best_accuracy = test_results['accuracy']
        torch.save(deepcoral.state_dict(), 'model_checkpoints/best_model.pth')
        print(f"New best model saved with accuracy: {best_accuracy:.2f}%")


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x10 and 512x10)

In [28]:
prms = {'device': 'cuda' if torch.cuda.is_available() else 'cpu'}

deepcoral = DeepCORAL()  # Replace with your actual model initialization
deepcoral.to(prms['device'])

# Load the best model
deepcoral.load_state_dict(torch.load('model_checkpoints/best_model.pth'))

test_results = test(deepcoral, svhn_testloader, epoch=1, params=prms)

print(test_results)

RuntimeError: Error(s) in loading state_dict for DeepCORAL:
	Missing key(s) in state_dict: "sharedNet.features.0.weight", "sharedNet.features.0.bias", "sharedNet.features.2.weight", "sharedNet.features.2.bias", "sharedNet.features.5.weight", "sharedNet.features.5.bias", "sharedNet.features.7.weight", "sharedNet.features.7.bias", "sharedNet.features.10.weight", "sharedNet.features.10.bias", "sharedNet.features.12.weight", "sharedNet.features.12.bias", "sharedNet.features.14.weight", "sharedNet.features.14.bias", "sharedNet.features.17.weight", "sharedNet.features.17.bias", "sharedNet.features.19.weight", "sharedNet.features.19.bias", "sharedNet.features.21.weight", "sharedNet.features.21.bias", "sharedNet.features.24.weight", "sharedNet.features.24.bias", "sharedNet.features.26.weight", "sharedNet.features.26.bias", "sharedNet.features.28.weight", "sharedNet.features.28.bias", "sharedNet.classifier.0.weight", "sharedNet.classifier.0.bias", "sharedNet.classifier.3.weight", "sharedNet.cla

In [ ]:
### UDA using SVHN pre-trained VGG16

In [ ]:
best_accuracy = 0.0
# Example of loading pretrained weights into DeepCORAL
pretrained_path = 'model_checkpoints/3channeled_vgg16_svhn.pth'  # Specify your pre-trained VGG-16 model path
deepcoral = DeepCORAL(num_classes=10)
deepcoral = load_pretrained(deepcoral.sharedNet, pretrained_path)
print(deepcoral)

# Move model to device
deepcoral.to(params['device'])

optimizer = optim.Adam(deepcoral.parameters(), lr=0.00002)

for epoch in range(1, 50 + 1):
    train_results = train(deepcoral, optimizer, epoch, _lambda=0.5, params=args)
    test_results = test(deepcoral, mnist_testloader, epoch, params=args)
    print(f"Test Epoch: {epoch}, Accuracy: {test_results['accuracy']:.2f}%")

    # Save the model if the accuracy improves
    if test_results['accuracy'] > best_accuracy:
        best_accuracy = test_results['accuracy']
        torch.save(deepcoral.state_dict(), 'model_checkpoints/best_model_svhn.pth')
        print(f"New best model saved with accuracy: {best_accuracy:.2f}%")

test_results_source = test(deepcoral, svhn_testloader, epoch=1, params=args)
print(test_results_source)

In [ ]:
prms = {'device': 'cuda' if torch.cuda.is_available() else 'cpu'}

# deepcoral = DeepCORAL()  # Replace with your actual model initialization
# deepcoral.to(prms['device'])

# Load the best model
# deepcoral.load_state_dict(torch.load('model_checkpoints/best_model_svhn.pth'))



print(test_results)